<a href="https://colab.research.google.com/github/adanfernandez/Data-Mining/blob/main/TextRank_SimHash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Rank & Simhash
## Preparación del entorno

In [ ]:
!pip install spacy==3.0.5
!pip install simhash
!pip install langdetect
!pip install shifterator
!pip install ndjson
!pip install pytextrank
!python -m spacy download es_core_news_lg

Descargamos los datos


In [ ]:
%cd /content
!mkdir limpio
!mkdir ejercicio2
%cd limpio

In [ ]:
!gdown --id 1brYy8Tiooo2Uw--FUmPb8lpkxJGM4cnR
!unzip tweets-limpios.ndjson.zip

## Cuerpo del ejercicio
Obtenemos los tweets por días. Esta operación tardará en torno a los 3 minutos y medio.

In [ ]:
from tqdm import tqdm
import time
import datetime
import io
import os
import ndjson
import math


SPANISH_STOPWORDS = {'a','al','algo','algunas','algunos','ante','antes','como',
                     'con','contra','cual','cuando','de','del','desde','donde',
                     'durante','e','el','él','ella','ellas','ellos','en','entre',
                     'era','erais','éramos','eran','eras','eres','es','esa',
                     'esas','ese','eso','esos','esta','está','estaba','estabais',
                     'estábamos','estaban','estabas','estad','estada','estadas',
                     'estado','estados','estáis','estamos','están','estando',
                     'estar','estará','estarán','estarás','estaré','estaréis',
                     'estaremos','estaría','estaríais','estaríamos','estarían',
                     'estarías','estas','estás','este','esté','estéis','estemos',
                     'estén','estés','esto','estos','estoy','estuve','estuviera',
                     'estuvierais','estuviéramos','estuvieran','estuvieras',
                     'estuvieron','estuviese','estuvieseis','estuviésemos',
                     'estuviesen','estuvieses','estuvimos','estuviste',
                     'estuvisteis','estuvo','fue','fuera','fuerais','fuéramos',
                     'fueran','fueras','fueron','fuese','fueseis','fuésemos',
                     'fuesen','fueses','fui','fuimos','fuiste','fuisteis','ha',
                     'habéis','había','habíais','habíamos','habían','habías',
                     'habida','habidas','habido','habidos','habiendo','habrá',
                     'habrán','habrás','habré','habréis','habremos','habría',
                     'habríais','habríamos','habrían','habrías','han','has',
                     'hasta','hay','haya','hayáis','hayamos','hayan','hayas',
                     'he','hemos','hube','hubiera','hubierais','hubiéramos',
                     'hubieran','hubieras','hubieron','hubiese','hubieseis',
                     'hubiésemos','hubiesen','hubieses','hubimos','hubiste',
                     'hubisteis','hubo','la','las','le','les','lo','los','más',
                     'me','mi','mí','mía','mías','mío','míos','mis','mucho',
                     'muchos','muy','nada','ni','no','nos','nosotras','nosotros',
                     'nuestra','nuestras','nuestro','nuestros','o','os','otra',
                     'otras','otro','otros','para','pero','poco','por','porque',
                     'que','qué','quien','quienes','se','sea','seáis','seamos',
                     'sean','seas','será','serán','serás','seré','seréis',
                     'seremos','sería','seríais','seríamos','serían',
                     'serías','sí','sido','siendo','sin','sobre','sois','somos',
                     'son','soy','su','sus','suya','suyas','suyo','suyos','también',
                     'tanto','te','tendrá','tendrán','tendrás','tendré','tendréis',
                     'tendremos','tendría','tendríais','tendríamos','tendrían',
                     'tendrías','tened','tenéis','tenemos','tenga','tengáis',
                     'tengamos','tengan','tengas','tengo','tenía','teníais',
                     'teníamos','tenían','tenías','tenida','tenidas','tenido',
                     'tenidos','teniendo','ti','tiene','tienen','tienes','todo',
                     'todos','tu','tú','tus','tuve','tuviera','tuvierais',
                     'tuviéramos','tuvieran','tuvieras','tuvieron','tuviese',
                     'tuvieseis','tuviésemos','tuviesen','tuvieses','tuvimos',
                     'tuviste','tuvisteis','tuvo','tuya','tuyas','tuyo','tuyos',
                     'un','una','uno','unos','vosotras','vosotros','vuestra',
                     'vuestras','vuestro','vuestros','y','ya','yo'}

def quitar_tildes_n_pasar_mayusculas(texto):
  texto = texto.lower()
  texto = texto.replace('á', 'a')
  texto = texto.replace('é', 'e')
  texto = texto.replace('í', 'i')
  texto = texto.replace('ó', 'o')
  texto = texto.replace('ú', 'u')
  texto = texto.replace('ñ', 'n')
  return texto


def tweets_dias(jsons, tweets_dias):
  for tuit in jsons:
    dtime = tuit['created_at']
    timestamp = time.mktime(datetime.datetime.strptime(dtime,'%a %b %d %H:%M:%S +0000 %Y').timetuple())
    timestamp=str(math.floor(timestamp/86400)*86400)
    if timestamp not in tweets_dias:
      tweets_dias[timestamp]=[]
      diccionario = {}
      print(timestamp)
    tuit_append = {}
    tuit_append['text'] = tuit['text']
    tuit_append['tokens'] =  list(filter(lambda x: len(str(x)) > 1 and x  not in {'http','https','www','com','tinyurl','html','twitter', 'rt', 'reuters', 'bbc', 'cnn'} and x not in SPANISH_STOPWORDS, tuit['tokens']))
    #tuit_append['tokens'] = tuit['tokens']
    tweets_dias[timestamp].append(tuit_append)
  return tweets_dias


def get_tweets():
  f = io.open("tweets-limpios.ndjson", mode="r", encoding="utf-8")
  tamano_leido=0
  tamano_bloque=20*1024*1024
  tamano = os.path.getsize("tweets-limpios.ndjson")

  segmentados = []
  tweets={}

  while True:
    content = f.readlines(tamano_bloque)
    content = " ".join(content) # readlines devuelve una lista, lo unimos en una cadena
    tamano_leido += tamano_bloque
    if not content:
      break
    else:
      jsons = ndjson.loads(content)
      tweets = tweets_dias(jsons, tweets)
  return tweets

tweets = get_tweets()


Se define una función para obtener n elementos aleatorios de una lista. En este caso elegimos 10 mil.

In [ ]:
import random

def get_random_from_list(n=10000):
  random_list = {}
  aux = n

  for key in tweets.keys():
    n = aux
    if n > len(tweets[key]):
      n=len(tweets[key])
    random_list[key] = random.sample(tweets[key], n)
  return random_list

random_tweets = get_random_from_list()

Obtenemos todos los tokens por día. Es decir, un token por cada clave (timestamp).

In [ ]:
tokens_dia = {}
for key in random_tweets.keys():
  tokens_dia[key] = []
  for tweet in random_tweets[key]:
    for token in tweet['tokens']:
        tokens_dia[key].append(token)


Aplicamos textRank a cada día. Es decir, se aplicará a la lista de elementos tokenizados que contiene cada día.

In [ ]:
import pytextrank
import spacy

def aplicar_textrank(texto, nlp):
  doc = nlp(texto)
  lemas = doc._.textrank.ranks
  lemas = sorted(lemas.items(), key=lambda x: x[1],reverse=True)
  resultados = {}
  resultados["lemas"] = lemas
  return resultados

max_caracteres = 1000000

def aplicar_textrank_tweets_tokenizados_dia():
  resultado = {}
  nlp = spacy.load("es_core_news_lg")
  nlp.add_pipe("textrank")
  with tqdm(total=len(tokens_dia.keys())) as barra:
    for key in tokens_dia.keys():
      barra.update(1)
      text =  " ".join(tokens_dia[key])[0:max_caracteres]
      resultado [key] = aplicar_textrank(text, nlp)
    return resultado

ranking = aplicar_textrank_tweets_tokenizados_dia()
print("El ranking ha terminado.")

Visualizamos el ranking de text rank.

In [ ]:
import pprint


for key in ranking:
  ranking_dia = ranking[key]
  for lema in ranking_dia["lemas"][0:30]:
    termino = lema[0]
    puntuacion = lema[1]
    print(termino.lemma, "\t", puntuacion)
    #if termino.pos=='NOUN':
     # print(termino.lemma, "\t", puntuacion)
  print("\n\n\n")

  

Tras esto, calcularemos el text rank de cada tweet, pudiendo sacar los mil tweets con mayor puntuación.

In [ ]:
puntuaciones = {}

for day in tweets:
  puntuaciones[day] = []
  for tweet in tweets[day]:
    puntuacion = 0.0
    ranking_dia = ranking[day]
    for lema in ranking_dia["lemas"][0:30]:
      if lema[0].lemma in tweet['tokens']:
        puntuacion += lema[1]
    tweet['puntuacion'] = 0.0
    if len(tweet['tokens']) != 0:
      tweet['puntuacion'] = puntuacion/(len(tweet['tokens']))
    puntuaciones[day].append(tweet)


Cogemos los mil primeros tweets.

In [ ]:
orden = {}
for day in puntuaciones.keys():
  orden[day] = sorted(puntuaciones[day], key=lambda x: x['puntuacion'], reverse=True)[0:1000]

Eliminamos duplicados aplicando simhash a los mil más relevantes

In [ ]:
from simhash import Simhash, SimhashIndex

def duplicados(tweets):
  firmas = []
  tweets_limpios  = []
  for complete_tweet in tweets:
    tweet = complete_tweet['text']
    firma = Simhash(tweet)
    firmas.append((tweet, firma))
  
  indice = SimhashIndex(firmas, k=10)
  indice.log.setLevel(0)
  for tweet in tweets:
    firma = Simhash(tweet['text'])
    duplicados = indice.get_near_dups(firma)
    for dup in duplicados:
      if dup not in tweets_limpios:
        tweets_limpios.append(dup)
      break
  return tweets_limpios
  
tweets_simhash={}
with tqdm(total=len(orden.keys())) as barra:
  for key in orden.keys():
    barra.update(1)
    tweets_simhash[key] = duplicados(orden[key])
  

In [ ]:
for ors in tweets_simhash.keys():
  print(len(tweets_simhash[ors]))

Visualizamos  los tweets por día...

In [ ]:
for day in orden:
  for tweet in tweets_simhash[day]:
    print(tweet)
print("\n\n\n\n\n\n\n\n\n\n\n OTRO DÍA...")